In [1]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Features Engineering and Selection

In [2]:
df = pd.read_csv("D:\data prep\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_bureau.csv")
df.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR
0,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,254629
1,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,254629
2,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,254629
3,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,254629
4,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,254629


In [3]:
# Bỏ những cột có correlation cao
#df = df.drop(columns=['CREDIT_CURRENCY','DAYS_ENDDATE_FACT','DAYS_CREDIT_UPDATE'])

# Merge with TARGET TABLE
df2 = pd.read_csv("D:\data prep\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_application_train.csv")
df2 = df2[['SK_ID_CURR','TARGET']]

merge_table = pd.merge(df2, df, on='SK_ID_CURR', how='left')

#Thêm feature num_past_loans để thể hiện số lượng khoản vay của mỗi khách hàng trong quá khứ
num_past_loan = df[['SK_ID_CURR', 'DAYS_CREDIT']].groupby('SK_ID_CURR')['DAYS_CREDIT'].count().reset_index()
num_past_loan.columns = ['SK_ID_CURR','PAST_LOAN_COUNT']

#Thêm feature num_of_type để thể hiện số lượng các loại khoản vay của mỗi khách hàng trong quá khứ
num_of_type = df[['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).nunique().reset_index()
num_of_type.columns = ['SK_ID_CURR','CREDIT_TYPE_COUNT']

#Thêm feature avt_loans để thể hiện những khoan vay chưa được trả hết
df['CREDIT_ACTIVE_BINARY'] = df['CREDIT_ACTIVE']

def check(status):
    if status == 'Closed':
        result = 0
    else:
        result = 1 
    return result

df['CREDIT_ACTIVE_BINARY'] = df.apply(lambda x: check(x.CREDIT_ACTIVE), axis = 1)

atv_loans = df.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})
atv_loans['ACTIVE_LOANS_PERCENTAGE'] = atv_loans['ACTIVE_LOANS_PERCENTAGE'] * 100
df = df.drop(columns=['CREDIT_ACTIVE_BINARY'])
atv_loans

#Thêm feature debt_credit để thể hiện tần suất sử dụng thẻ debt-to-credit
data_with_sum_debt_pos_sign_all = df[df['AMT_CREDIT_SUM_DEBT']>0]

#chỉ consider những khoản debt đã đc trả hoàn toàn.
grp1 = data_with_sum_debt_pos_sign_all[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
grp2 = df[['SK_ID_CURR', 'AMT_CREDIT_SUM']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM': 'TOTAL_CUSTOMER_CREDIT'})

df = df.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
df = df.merge(grp2, on = ['SK_ID_CURR'], how = 'left')
del grp1, grp2


df['DEBT_CREDIT_RATIO'] = (df['TOTAL_CUSTOMER_DEBT']/df['TOTAL_CUSTOMER_CREDIT'])*100
df['DEBT_CREDIT_RATIO'] = df['DEBT_CREDIT_RATIO'].replace([np.inf, -np.inf], np.nan)

del df['TOTAL_CUSTOMER_DEBT'], df['TOTAL_CUSTOMER_CREDIT']

debt_credit = df.groupby(by = ['SK_ID_CURR'])['DEBT_CREDIT_RATIO'].mean().reset_index().rename(index=str)
debt_credit

#Count Overdue Counts
def overDue(df):
    name_flag = 'OVERDUE_FLAG'
    name = 'OVERDUE'
    
    def fun(x):
        days = abs(x)
        if days > 0:
            return 1
        else:
            return 0

    df[name_flag] = df['CREDIT_DAY_OVERDUE'].apply(fun)
    
    df[name] = df.groupby('SK_ID_CURR')[name_flag].transform('sum')
    
    #lets increase the groups by 1 to make the base group 1
    df[name] = df[name].apply(lambda x:x+1)
    
    #drop column
    df.drop([name_flag],axis=1, inplace=True)

#apply function
overDue(df)

#add features
final_dataset = df.merge(num_past_loan,
                                      how='left', 
                                      on='SK_ID_CURR')
final_dataset = final_dataset.merge(num_of_type,
                                      how='left', 
                                      on='SK_ID_CURR')
final_dataset = final_dataset.merge(atv_loans,
                                      how='left', 
                                      on='SK_ID_CURR')
final_dataset = final_dataset.merge(debt_credit,
                                      how='left', 
                                      on='SK_ID_CURR')

final_dataset = final_dataset.drop(columns = ['DEBT_CREDIT_RATIO_y'])

final_dataset = final_dataset.rename(columns = {'DEBT_CREDIT_RATIO_x': 'DEBT_CREDIT_RATIO'})

#Merge With Bureau Balance
df3 = pd.read_csv("D:\data prep\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_bureau_balance.csv")
df3.head()

def extract_mean(x):
    y = x.groupby('SK_ID_BUREAU', as_index=False).mean().add_prefix('BUR_BAL_MEAN_') 
    return y

# apply formula to create bureau_balance dataframe grouped by SK_ID_BUREAU with mean values of all numerical columns
bureau_bal_mean = extract_mean(df3)

bureau_bal_mean = bureau_bal_mean.rename(columns = {'BUR_BAL_MEAN_SK_ID_BUREAU' : 'SK_ID_BUREAU'})

final_dataset = final_dataset.merge(bureau_bal_mean,  on = 'SK_ID_BUREAU', how = 'left')
final_dataset.drop('SK_ID_BUREAU', axis = 1, inplace = True) # we don't need this internal ID anymore

def extract_mean_bur(x):
    y = x.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('BUR_MEAN_') # note that we have changed the ID to group by and the prefix to add
    return y

final_data_mean_values = extract_mean_bur(final_dataset)
final_data_mean_values = final_data_mean_values.rename(columns = {'BUR_MEAN_SK_ID_CURR' : 'SK_ID_CURR'})

C:\Users\Admin\AppData\Local\Temp\ipykernel_10496\1291571864.py:103: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  y = x.groupby('SK_ID_BUREAU', as_index=False).mean().add_prefix('BUR_BAL_MEAN_')
C:\Users\Admin\AppData\Local\Temp\ipykernel_10496\1291571864.py:115: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  y = x.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('BUR_MEAN_') # note that we have changed the ID to group by and the prefix to add


In [4]:
final_data_mean_values

,SK_ID_CURR,BUR_MEAN_DAYS_CREDIT,BUR_MEAN_CREDIT_DAY_OVERDUE,BUR_MEAN_DAYS_CREDIT_ENDDATE,BUR_MEAN_DAYS_ENDDATE_FACT,BUR_MEAN_AMT_CREDIT_MAX_OVERDUE,BUR_MEAN_CNT_CREDIT_PROLONG,BUR_MEAN_AMT_CREDIT_SUM,BUR_MEAN_AMT_CREDIT_SUM_DEBT,BUR_MEAN_AMT_CREDIT_SUM_LIMIT,BUR_MEAN_AMT_CREDIT_SUM_OVERDUE,BUR_MEAN_DAYS_CREDIT_UPDATE,BUR_MEAN_AMT_ANNUITY,BUR_MEAN_DEBT_CREDIT_RATIO,BUR_MEAN_OVERDUE,BUR_MEAN_PAST_LOAN_COUNT,BUR_MEAN_CREDIT_TYPE_COUNT,BUR_MEAN_ACTIVE_LOANS_PERCENTAGE,BUR_MEAN_BUR_BAL_MEAN_MONTHS_BALANCE
0,0,-63.000000,0.0,237.000000,NaN,NaN,0.0,8.551350e+04,77566.500000,NaN,0.0,-28.000000,NaN,90.706731,1.0,1.0,1.0,100.000000,NaN
1,1,-2348.000000,0.0,-2044.000000,NaN,11666.385,0.0,2.835000e+04,0.000000,0.0000,0.0,-18.000000,NaN,NaN,1.0,1.0,1.0,100.000000,-50.500000
2,2,-810.333333,0.0,-572.166667,-1014.000000,3771.990,0.0,4.239018e+04,16069.500000,0.0000,0.0,-625.500000,NaN,37.908544,1.0,6.0,1.0,33.333333,NaN
3,3,-1131.428571,0.0,-658.714286,-1394.000000,0.000,0.0,6.913607e+04,28395.642857,0.0000,0.0,-805.000000,NaN,41.072109,1.0,7.0,2.0,42.857143,NaN
4,4,-773.333333,0.0,474.500000,-761.000000,NaN,0.0,4.704240e+05,209084.250000,0.0000,0.0,-63.333333,NaN,44.445915,1.0,6.0,2.0,50.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263486,307504,-1149.750000,0.0,-557.750000,-987.000000,0.000,0.0,2.165383e+05,29941.875000,0.0000,0.0,-742.750000,NaN,13.827518,1.0,4.0,1.0,25.000000,NaN
263487,307505,-461.500000,0.0,5931.500000,NaN,0.000,0.0,1.809000e+06,0.000000,540000.0000,0.0,-12.000000,90463.50000,NaN,1.0,2.0,2.0,100.000000,-7.250000
263488,307506,-1390.500000,0.0,-1115.500000,-1095.250000,NaN,0.0,6.481575e+04,0.000000,0.0000,0.0,-862.500000,NaN,NaN,1.0,4.0,1.0,0.000000,NaN
263489,307508,-765.428571,0.0,587.666667,-318.333333,0.000,0.0,7.866153e+05,41958.000000,300.7875,0.0,-154.571429,6369.06375,4.571994,1.0,7.0,3.0,57.142857,-18.142857


In [5]:
final_data_mean_values.to_csv('bureau_balance_merge_bureau.csv', index = False)